In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
sector = "healthcare"

In [3]:
df_analysis = pd.read_csv(sector + "_dataset.csv")
df_analysis

,Date,Tick,Close,Polarity,News
0,2016-01-04,AAAP,30.670000,0.000000,0
1,2016-01-05,AAAP,30.880000,0.000000,0
2,2016-01-06,AAAP,29.960000,0.000000,0
3,2016-01-07,AAAP,29.650000,0.000000,0
4,2016-01-08,AAAP,28.000000,0.000000,0
5,2016-01-09,AAAP,27.596667,0.000000,0
6,2016-01-10,AAAP,27.193333,0.000000,0
7,2016-01-11,AAAP,26.790000,0.000000,0
8,2016-01-12,AAAP,24.710000,0.000000,0
9,2016-01-13,AAAP,26.090000,0.000000,0


<h3> Pivoting and Generating Features & Labels </h3>

In [11]:
df_analysis['Signals'] = [[x, y] for x,y in zip(df_analysis['Close'], df_analysis['Polarity'])]
df_analysis.head()

,Date,Tick,Close,Polarity,News,Signals
0,2016-01-04,AAAP,30.67,0.0,0,"[30.67, 0.0]"
1,2016-01-05,AAAP,30.88,0.0,0,"[30.88, 0.0]"
2,2016-01-06,AAAP,29.96,0.0,0,"[29.96, 0.0]"
3,2016-01-07,AAAP,29.65,0.0,0,"[29.65, 0.0]"
4,2016-01-08,AAAP,28.00,0.0,0,"[28.0, 0.0]"


In [16]:
#df_pivot = pd.pivot_table(df_analysis, values='Signals', index='Tick', columns='Date')
df_pivot_stock = pd.pivot_table(df_analysis, values='Close', index='Tick', columns='Date')
df_pivot_news = pd.pivot_table(df_analysis, values='Polarity', index='Tick', columns='Date')
df_pivot_stock.head(), df_pivot_stock.tail()

(Date  2016-01-04  2016-01-05  2016-01-06  2016-01-07  2016-01-08  2016-01-09  \
 Tick                                                                           
 AAAP       30.67       30.88       29.96       29.65       28.00   27.596667   
 ABBV       57.61       57.37       57.38       57.21       55.65   55.060000   
 ABC       101.87      103.36      101.78       98.53       97.96   96.660000   
 ABMD       85.24       85.00       85.30       81.92       84.58   84.720000   
 ACAD       34.62       34.46       29.24       28.73       28.04   27.730000   
 
 Date  2016-01-10  2016-01-11  2016-01-12  2016-01-13     ...      2017-09-21  \
 Tick                                                     ...                   
 AAAP   27.193333       26.79       24.71       26.09     ...           57.15   
 ABBV   54.470000       53.88       54.84       51.18     ...           87.41   
 ABC    95.360000       94.06       94.18       93.14     ...           80.38   
 ABMD   84.860000       85

<h3> Prepare input for Tensor Flow </h3>

In [18]:
window = 100
features = []
labels = []
perc_var = []
dates = []

i = 0
end_col = window + 1

while end_col + i < df_pivot_stock.shape[1]:
    features.append(np.array([df_pivot_stock.iloc[:, i:end_col+i-1].values, df_pivot_news.iloc[:, i:end_col+i-1].values]))
    l = (df_pivot_stock.iloc[:,end_col+i] - df_pivot_stock.iloc[:,end_col+i-1]).map(lambda x: 1 if x>0 else 0).values
    labels.append(l)
    
    perc_var.append( ((df_pivot_stock.iloc[:,end_col+i] - df_pivot_stock.iloc[:,end_col+i-1]) / df_pivot_stock.iloc[:,end_col+i-1]).values )
    dates.append(df_pivot_stock.columns[end_col+i])
    i = i + 1

features = np.array(features)
labels = np.array(labels)
perc_var = np.array(perc_var)
features.shape, labels.shape, features[0], labels[0:2], perc_var.shape, perc_var[0], dates[-100:]

((535, 2, 154, 100),
 (535, 154),
 array([[[  30.67      ,   30.88      ,   29.96      , ...,   35.76666667,
            35.8       ,   35.75      ],
         [  57.61      ,   57.37      ,   57.38      , ...,   58.49      ,
            58.5       ,   59.89      ],
         [ 101.87      ,  103.36      ,  101.78      , ...,   84.73      ,
            84.44      ,   84.9       ],
         ..., 
         [  58.86      ,   60.06      ,   59.19      , ...,   59.57333333,
            59.5       ,   59.71      ],
         [ 101.81      ,  103.93      ,  104.38      , ...,  109.18666667,
           109.3       ,  111.57      ],
         [  47.27      ,   48.01      ,   48.02      , ...,   46.76      ,
            46.67      ,   47.01      ]],
 
        [[   0.        ,    0.        ,    0.        , ...,    0.        ,
             0.        ,    0.        ],
         [   0.        ,    0.        ,    0.        , ...,    0.        ,
             0.        ,    0.        ],
         [   0.     

<h3> Split Train - Eval </h3>

In [19]:
train_features = features[:-100]
test_features = features[-100:]
train_labels = labels[:-100]
test_labels = labels[-100:]

train_features.shape, test_features.shape, train_labels.shape, test_labels.shape

((435, 2, 154, 100), (100, 2, 154, 100), (435, 154), (100, 154))

<h3> TRAIN CNN </h3>

In [20]:
exec(open("cnn_stocks.py").read())

In [21]:
# Create the Estimator
stocks_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="../tmp/stocks_convnet_model")

# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '../tmp/stocks_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001334A763198>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_features},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

stocks_classifier.train(
    input_fn=train_input_fn,
    steps=1000,
    hooks=[logging_hook])

Tensor("Reshape:0", shape=(100, 154, 100, 2), dtype=float32)
Tensor("conv2d/Relu:0", shape=(100, 154, 100, 32), dtype=float32)
Tensor("max_pooling2d/MaxPool:0", shape=(100, 154, 50, 32), dtype=float32)
Tensor("conv2d_2/Relu:0", shape=(100, 154, 50, 8), dtype=float32)
Tensor("max_pooling2d_2/MaxPool:0", shape=(100, 154, 10, 8), dtype=float32)
Tensor("conv2d_3/Relu:0", shape=(100, 154, 10, 2), dtype=float32)
Tensor("max_pooling2d_3/MaxPool:0", shape=(100, 154, 5, 2), dtype=float32)
Tensor("Reshape_1:0", shape=(100, 1540), dtype=float32)
Tensor("dense/Relu:0", shape=(100, 512), dtype=float32)
Tensor("dropout/dropout/mul:0", shape=(100, 512), dtype=float32)
Tensor("dense_2/BiasAdd:0", shape=(100, 154), dtype=float32)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ../tmp/stocks_convnet_model\model.ckpt-1000
INFO:tensorflow:Saving checkpoints for 1001 into ../tmp/stocks_convnet_model\model.ckpt.
INFO:tensorflow:probabilities = [[ 0.0065141   0.00626917 

<h3> Model Evaluation </h3>

In [ ]:
import datetime as dt
first_saturday = dt.datetime.strptime("2017-06-24", "%Y-%m-%d")

holidays_2017 = ['2017-01-02',
                 '2017-01-16',
                 '2017-02-20',
                 '2017-04-14',
                 '2017-05-29',
                 '2017-07-04',
                 '2017-09-04',
                 '2017-11-23',
                 '2017-12-25']

eval_days = [x not in holidays_2017 and (dt.datetime.strptime(x, "%Y-%m-%d") - first_saturday).days % 7 > 1 
     for x in dates[-100:]]

In [ ]:
perc_var_eval = perc_var[-100:]

new_perc_val = []

finished = False
i = 0
while i < len(perc_var_eval):
    print (eval_days[i])
    if eval_days[i]:
        new_perc_val.append(perc_var_eval[i])
        i = i + 1
    else:
        cumulative = 0
        while(not eval_days[i]):
            cumulative = perc_var_eval[i]
            new_perc_val.append(np.array([0 for k in perc_var_eval[i]]))
            i = i + 1
            if i >= len(eval_days):
                finished = True
                break;
        if not finished:
            new_perc_val.append(perc_var_eval[i] + cumulative)
            i = i + 1

perc_var_eval = new_perc_val

In [ ]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": test_features},
    y=test_labels,
    num_epochs=1,
    shuffle=False)

earnings = []
for N in range(1, 155):
    eval_results = stocks_classifier.predict(input_fn=eval_input_fn)
    
    capital = 1
    for i, r in enumerate(eval_results):
        top_n = r['probabilities'].argsort()[-N:][::-1]
        allocation = [0 for x in perc_var_eval[i]]
        for j, t in enumerate(top_n):
            allocation[t] = capital / N

        variation = allocation + (allocation * perc_var_eval[i])
        capital = variation.sum()
    earnings.append(capital)

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

plt.figure(figsize=[16,8])
plt.plot([i for i in range(1,155)], [(e-1)*100 for e in earnings])
plt.plot([i for i in range(1,155)], [(earnings[-1]-1)*100 for j in range(154)])
plt.plot([i for i in range(1,155)], [0 for j in range(154)], 'r--')
plt.xticks([i for i in range(1, 155) if (i)%5 == 0 or i==1])

#plt.annotate(str((max(earnings)-1)*100 + "% with {} stocks".format(earnings.index(max(earnings))+1)), xy=(earnings.index(max(earnings))+1, (max(earnings)-1)*100)) 

plt.title("Performance Analysis")
plt.ylabel("% Gain/Loss")
plt.xlabel("# Picked Stocks")
plt.grid(True)
plt.savefig('performance.eps', format='eps')

In [ ]:
max(earnings), min(earnings), earnings[-1], (max(earnings)-1)/(earnings[-1] - 1)

In [ ]:
earnings.index(max(earnings)) + 1